In [1]:
!nvidia-smi

Sun Dec  8 11:25:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:C1:00.0 Off |                    0 |
| N/A   31C    P0             45W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
############# Load Template Sentences from Chroma DB ####################################

In [3]:
occupation = "politician"
CHROMA_SOURCE = "template_" + occupation
CHROMA_SOURCE_JSON = "json_" + occupation
CHROMA_CHUNK_SIZE = 1000
CHROMA_PERSIST_DIRECTORY = "./chroma_db"
CHROMA_COLLECTION_NAME = occupation + "_json_files"
EMBEDDINGS_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [4]:
import os
os.environ["HF_HOME"] = "~/scratch/hf-cache"
from langchain.docstore.document import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
############# Load LLama ######################

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
with torch.no_grad():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = "<YOUR HF TOKEN>", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/hice1/tkaple3/.conda/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.256 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 6/6 [05:37<00:00, 56.30s/it]


In [ ]:
########## Model HyperParameters #######################

In [6]:
gen_config = model.generation_config
print(gen_config)

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "max_length": 131072,
  "pad_token_id": 128004,
  "temperature": 0.6,
  "top_p": 0.9
}



In [ ]:
################ Prompt Template ########################

In [7]:
my_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [10]:
######### Politicians to Query For ######################
all_entities = [['Wilfred', 'Elrington'], ['Ernst', 'Siemens'], ['Erich', 'Ludendorff'], ['Ernst', 'Röhm'], ['Daniel', 'Romanowitsch'], ['Franz', 'Müntefering'], ['Hans', 'Fritzsche'], ['Karl', 'Gebhardt'], ['Robert', 'Ley'], ['Otto', 'Braun'], ['Oskar', 'Dirlewanger'], ['Mikhail', 'Lomonosov'], ['Hanns', 'Schleyer'], ['Sandra', 'Perković'], ['Friedrich', 'von Humboldt'], ['Ludwig', 'Grimm'], ['Jean-Marie', 'Le Pen'], ['Zheng', 'Ying'], ['Rosa', 'Luxemburg'], ['Giacomo', 'Puccini'], ['Giuseppe', 'Verdi'], ['Jacques', 'Cartier'], ['Yury', 'Gagarin'], ['Lun', 'Sima'], ['Margaret', 'Roberts'], ['Jian', 'Yang'], ['Nina', 'Andreeva'], ['Irène', 'Joliot-Curie'], ['Willoughby', 'Newton'], ['Charles', 'Maurras'], ['Guy', 'Teissier'], ['William', 'Terry'], ['James', 'Gibson'], ['Allen', 'Sheppard'], ['Elisha', 'Meredith'], ['Henry', 'Alexander'], ['Todd', 'Platts'], ['David', 'Cairns'], ['Marit', 'Paulsen'], ['Michael', 'Shaw'], ['John', 'Gregg'], ['Leonid', 'Razvozzhayev'], ['Mohamed', 'Sheikh'], ['Albert', 'Grimaldi'], ['Jacqueline', 'Cochran'], ['V.', 'Kalairajan'], ['Louis', 'Aragon'], ['Volker', 'Beck'], ['Blaise', 'Compaoré'], ['Karl', 'Frank'], ['Danilo', 'Türk'], ['Dimitris', 'Christofias'], ['Ali', 'Khamenei'], ['Carlos', 'Funes'], ['Pierre', 'Nkurunziza'], ['Joseph', 'Fischer'], ['Ernst', 'Arndt'], ['Gustav', 'Noske'], ['Yahya', 'Jammeh'], ['Andry', 'Rajoelina'], ['Thomas', 'de Maizière'], ['Emomali', 'Rakhmonov'], ['Yoweri', 'Museveni'], ['Michael', 'Higgins'], ['Traian', 'Băsescu'], ['Jacob', 'Zuma'], ['Toomas', 'Ilves'], ['Abdullah', 'House of Saud'], ['Bamir', 'Topi'], ['Hans-Gert', 'Pöttering'], ['Karolos', 'Papoulias'], ['Jalal', 'Talabani'], ['Abdelaziz', 'Bouteflika'], ['Erich', 'Mielke'], ['Juan Manuel', 'Santos'], ['Sihamoni', 'Norodom'], ['Laura', 'Chinchilla'], ['Otto', 'Gessler']]

In [13]:
import time
import json
import ast
import time
from tqdm import tqdm
error = 0
SAVE_TO_DISK = 1
with open("Analysis_Article_Generation_Without_Template_v2.json", "w") as f:
    for entry in tqdm(all_entities):
        first_name = entry[0]
        last_name = entry[1]
        entity_name = first_name + " " + last_name
        print("Your are querying for: "+ entity_name)
    ################# Get JSON from Chroma Cache ###########################
        start_time = time.time()
        target_collection_name = occupation + "_json_files" 
        embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
        print("Target Collection Is: " + target_collection_name)
        chroma_db_from_disk = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,                             
                                    collection_name=target_collection_name,
                                    embedding_function=embeddings)
        results = chroma_db_from_disk.get(
            where={
                "$and": [
                    {"first_name": first_name},
                    {"last_name": last_name}
                ]
            },
            include=["documents", "metadatas"])
        documents = [
            Document(page_content=doc, metadata=meta)
            for doc, meta in zip(results["documents"], results["metadatas"])
        ]
        end_time = time.time()
        chroma_json_retrieval_time = end_time - start_time
        print(f"Chroma Retrieval Time: {chroma_json_retrieval_time:.4f} seconds")
        document = documents[0]
        content = ast.literal_eval(document.page_content)[0]
        response_start = content.index("### Response:")
        json_start = content.index("{", response_start)
        json_end = content.rindex("}")
        json_string = content[json_start:json_end+1]
        try:
            parsed_dict = ast.literal_eval(json_string)
            json_data = json.dumps(parsed_dict, ensure_ascii=False, indent=2)
            #print(json_data)
        except (SyntaxError, ValueError) as e:
            error += 1
            print(f"Error parsing JSON: {e}")
            continue
    ################ Inference for Article Generation ######################
        start_time = time.time()
        with torch.no_grad():
            FastLanguageModel.for_inference(model)
            inputs = tokenizer(
            [
                my_prompt.format(
                    f"Generate an Article in Hindi Language for {entity_name} using the details given in the json string below for this person. Do not any other extra English text after the article.", # instruction
                    f"JSON:\n{json_data}", # input
                    "", # output - leave this blank for generation!
                )
            ], return_tensors = "pt").to("cuda")
            print("Number of Tokens in Input: " + str(inputs['input_ids'].shape[1]))
            outputs = model.generate(**inputs, max_new_tokens = 8192)
            print("Number of Tokens in output: " + str(outputs.shape[1]))
            final_article = tokenizer.batch_decode(outputs)
            #print(final_article)
        end_time = time.time()
        article_generation_time = end_time - start_time
        print(f"Article Generation Time: {article_generation_time:.4f} seconds")
    ################## CHROMA SAVE ############################
        start_time = time.time()
        target_collection_name = occupation + "_generated_articles_without_template"
        if SAVE_TO_DISK == 1:
            docs_2 = [Document(page_content=str(final_article), metadata={"source": f"{occupation}_generated_articles_without_template", "first_name": first_name, "last_name": last_name })]
            text_splitter = CharacterTextSplitter(chunk_size=CHROMA_CHUNK_SIZE, chunk_overlap=0)
            docs_2 = text_splitter.split_documents(docs_2)
            embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
            local_chroma = Chroma.from_documents(docs_2, embeddings, persist_directory=CHROMA_PERSIST_DIRECTORY, collection_name=target_collection_name)
            print("Your document is Saved to Chroma")
        else:
            print("WARNING: YOU CHOOSE NOT TO SAVE THE ARTICLE TO CHROMA") 
        end_time = time.time()
        chroma_save_time = end_time - start_time
        print(f"Execution time: {chroma_save_time:.4f} seconds")
        op = { "First Name": first_name, "Last Name": last_name, "Time for JSON Retrieval": chroma_json_retrieval_time,
              "Time For Chroma Write": chroma_save_time, "Input Tokens": inputs['input_ids'].shape[1], 
              "Output Tokens": outputs.shape[1], "Time For Article Generation": article_generation_time }
        json.dump(op, f, indent=4)

  0%|          | 0/78 [00:00<?, ?it/s]

Your are querying for: Wilfred Elrington
Target Collection Is: politician_json_files
Chroma Retrieval Time: 6.9510 seconds
Number of Tokens in Input: 595
Number of Tokens in output: 964
Article Generation Time: 38.7990 seconds


  1%|▏         | 1/78 [00:48<1:02:20, 48.58s/it]

Your document is Saved to Chroma
Execution time: 2.8246 seconds
Your are querying for: Ernst Siemens
Target Collection Is: politician_json_files
Chroma Retrieval Time: 1.5420 seconds
Number of Tokens in Input: 2173
Number of Tokens in output: 2758
Article Generation Time: 64.4814 seconds


  3%|▎         | 2/78 [01:56<1:15:55, 59.94s/it]

...

Your document is Saved to Chroma
Execution time: 2.2517 seconds
Your are querying for: Laura Chinchilla
Target Collection Is: politician_json_files
Chroma Retrieval Time: 1.6502 seconds
Number of Tokens in Input: 908
Number of Tokens in output: 1640
Article Generation Time: 71.8533 seconds


 99%|█████████▊| 77/78 [1:27:49<00:54, 54.66s/it]

Your document is Saved to Chroma
Execution time: 1.5119 seconds
Your are querying for: Otto Gessler
Target Collection Is: politician_json_files
Chroma Retrieval Time: 2.3097 seconds
Number of Tokens in Input: 1104
Number of Tokens in output: 1418
Article Generation Time: 30.9411 seconds


100%|██████████| 78/78 [1:28:24<00:00, 68.00s/it]

Your document is Saved to Chroma
Execution time: 1.9394 seconds
